In [43]:
import numpy as np
import sys
import os

# Adjust the path to point to external/AlphaPEM
sys.path.append(os.path.abspath("../external/AlphaPEM"))
# Importing constants' value and functions
from configuration.settings import current_density_parameters, physical_parameters, \
                                   computing_parameters, operating_inputs
from model.AlphaPEM import AlphaPEM

In [49]:
def build_fixed_parameters():
    type_current="polarization"
    type_fuel_cell="EH-31_2.0"
    t_step, i_step, delta_pola, i_EIS, ratio_EIS, f_EIS, t_EIS, current_density = current_density_parameters(type_current)
    # Operating conditions
    *_, i_max_pola = operating_inputs(type_fuel_cell)
    
    # Physical parameters
    Hcl, epsilon_mc, tau, Hmem, Hgdl, epsilon_gdl, epsilon_c, Hgc, Wgc, Lgc, Aact, e, Re, i0_c_ref, kappa_co, \
        kappa_c, a_slim, b_slim, a_switch, C_scl = physical_parameters(type_fuel_cell)
    # Computing parameters
    max_step, n_gdl, t_purge = computing_parameters(type_current, Hgdl, Hcl)

    return {
        "t_step": t_step,
        "i_step": i_step,
        "delta_pola": delta_pola,
        "i_EIS": i_EIS,
        "ratio_EIS": ratio_EIS,
        "f_EIS": f_EIS,
        "t_EIS": t_EIS,
        "current_density": current_density,
        "max_step": max_step,
        "n_gdl": n_gdl,
        "t_purge": t_purge,
        "type_fuel_cell": "manual_setup", 
        "type_current": "polarization",
        "type_auxiliary": "no_auxiliary",
        "type_control": "no_control",
        "type_purge": "no_purge",
        "type_display": "no_display",
        "type_plot": "fixed",
        "C_scl": C_scl,
        "i_max_pola": i_max_pola,
        "Aact": Aact,
        "Hgdl": Hgdl,
        "Hmem": Hmem,
        "Hcl": Hcl,
        "Hgc": Hgc,
        "Wgc": Wgc,
        "Lgc": Lgc
    }

In [80]:
parameter_ranges = {
    # Operating conditions
    "Tfc": (333, 363),  # Cell temperature (K)
    "Pa_des": (1.1e5, 3e5),  # Desired cell pressure (bara)
    "Pc_des": None,  # Desired cell pressure (bara)
    "Sa": (1.1, 3),  # Stoichiometry anode
    "Sc": (1.1, 10),  # Stoichiometry cathode
    "Phi_a_des": (0.1, 1),  # Desired entrance humidity anode
    "Phi_c_des": (0.1, 1),  # Desired entrance humidity cathode

    # Undetermined physical parameters
    "epsilon_gdl": (0.55, 0.8),  # GDL porosity
    "tau": (1.0, 4.0),  # Pore structure coefficient
    "epsilon_mc": (0.15, 0.4),  # CL volume fraction of ionomer
    "epsilon_c": (0.15, 0.3),  # GDL compression ratio
    "e": [3, 4, 5],  # Capillary exponent (should be an integer)
    "Re": (5e-7, 5e-6),  # Electron conduction resistance (Ω·m²)
    "i0_c_ref": (0.001, 500),  # Cathode exchange current density (A/m²)
    "kappa_co": (0.01, 40),  # Crossover correction coefficient (mol/(m·s·Pa))
    "kappa_c": (0, 100),  # Overpotential correction exponent
    "a_slim": (0, 1),  # Slim coefficient a
    "b_slim": (0, 1),  # Slim coefficient b
    "a_switch": (0, 1),  # Slim switch parameter
}

def sample_parameters(n_samples=100, parameter_ranges=parameter_ranges):
    samples = {}
    
    for key, val in parameter_ranges.items():
        if key == 'Pa_des':
            low, high = val
            samples['Pa_des'] = np.random.uniform(low, high, n_samples)
            low, high = (np.maximum(1, samples['Pa_des'] - 0.5), np.maximum(1, samples['Pa_des'] - 0.1))
            samples['Pc_des'] = np.random.uniform(low, high)

        elif isinstance(val, tuple):  # Continuous range
            low, high = val
            samples[key] = np.random.uniform(low, high, n_samples)

        elif isinstance(val, list):  # Discrete
            samples[key] = np.random.choice(val, n_samples)
        
    return samples

print(sample_parameters(n_samples=1))
sampled_parameters = sample_parameters(n_samples=1)
fixed_parameters = build_fixed_parameters()
combined_parameters = {**sampled_parameters, **fixed_parameters}

{'Tfc': array([356.60285168]), 'Pa_des': array([155812.39353818]), 'Pc_des': array([155812.26367209]), 'Sa': array([1.4735175]), 'Sc': array([9.94269637]), 'Phi_a_des': array([0.1015352]), 'Phi_c_des': array([0.27595478]), 'epsilon_gdl': array([0.66815998]), 'tau': array([1.20870533]), 'epsilon_mc': array([0.21905246]), 'epsilon_c': array([0.20627742]), 'e': array([5]), 'Re': array([2.97035011e-06]), 'i0_c_ref': array([406.83206174]), 'kappa_co': array([20.33682479]), 'kappa_c': array([3.4486891]), 'a_slim': array([0.88730565]), 'b_slim': array([0.07871217]), 'a_switch': array([0.0794422])}


In [81]:
def convert_arrays_to_numbers(d):
    new_dict = {}
    for key, value in d.items():
        new_dict[key] = float(value)
    return new_dict

sampled_parameters = convert_arrays_to_numbers(sampled_parameters)
combined_parameters = {**sampled_parameters, **fixed_parameters}

C:\Users\dejvi\AppData\Local\Temp\ipykernel_9576\562995038.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_dict[key] = float(value)


In [82]:
Simulator = AlphaPEM(**combined_parameters)

In [ ]:
import pandas as pd

max_len = max(len(lst) for lst in Simulator.variables.values())

# Pad all to max_len
for k in Simulator.variables:
    Simulator.variables[k] += [None] * (max_len - len(Simulator.variables[k]))

pd.DataFrame(Simulator.variables)
        

,C_v_agc,C_v_agdl_1,C_v_agdl_2,C_v_agdl_3,C_v_agdl_4,C_v_agdl_5,C_v_agdl_6,C_v_agdl_7,C_v_agdl_8,C_v_agdl_9,...,t,Ucell,S_sorp_acl,S_sorp_ccl,J_lambda_mem_acl,J_lambda_mem_ccl,Pagc,Pcgc,Phi_a_des,Phi_c_des
0,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,...,0.000000,0.957784,0.000000,0.000000,0.000000e+00,0.000000e+00,200000.000000,200000.000000,None,None
1,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,...,0.000002,0.957783,-0.000775,-0.002964,1.934419e-08,-7.417817e-08,200000.000014,200000.000000,None,None
2,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,...,0.000005,0.957783,-0.001547,-0.005907,3.868511e-08,-1.483523e-07,200000.000028,200000.000000,None,None
3,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,...,0.000030,0.957781,-0.009210,-0.034612,2.319279e-07,-8.898436e-07,200000.000168,200000.000000,None,None
4,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,6.395155,...,0.000055,0.957780,-0.016833,-0.062887,4.250283e-07,-1.631072e-06,200000.000289,200000.000000,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19223,11.073532,11.104827,11.111974,11.119113,11.126243,11.133365,11.140479,11.147584,11.154680,11.161769,...,1919.628121,0.277055,-1313.914488,-14891.115293,-1.234164e-02,-1.233356e-02,200000.000812,200000.017922,None,None
19224,11.073482,11.104777,11.111924,11.119063,11.126193,11.133315,11.140428,11.147533,11.154630,11.161718,...,1919.728121,0.277035,-1313.902093,-14891.372227,-1.234142e-02,-1.233313e-02,200000.000830,200000.018398,None,None
19225,11.073431,11.104726,11.111872,11.119011,11.126141,11.133263,11.140376,11.147481,11.154578,11.161666,...,1919.828121,0.277015,-1313.889237,-14891.635998,-1.234119e-02,-1.233269e-02,200000.000848,200000.018885,None,None
19226,11.073378,11.104673,11.111820,11.118958,11.126088,11.133210,11.140323,11.147428,11.154524,11.161613,...,1919.928121,0.276994,-1313.875908,-14891.906779,-1.234095e-02,-1.233224e-02,200000.000867,200000.019385,None,None
